In [18]:
import warnings
warnings.filterwarnings("ignore")
# Import all libraries
import spacy
import scispacy
from scispacy.linking import EntityLinker
import pandas as pd
from IPython.display import display  

# Load CSV containing UMLS_CUI and icd10 code mappings
umls_csv_path = "umls_cuis.csv"  # Update with your actual UMLS-ICD10 mapping CSV path
code_df = pd.read_csv(umls_csv_path, encoding="latin1")
print("UMLS-ICD10 File loaded successfully!")

# df_transformed will be our input for the model, for testing purpose only
df_transformed_path = "structured_diagnosis"
df_transformed = pd.read_csv(df_transformed_path)

#df_transformed = df_transformed.sample(n=10, random_state=42) 

df_transformed['discharge_diag'] = df_transformed['discharge_diag'].astype(str)

print("loaded diagnosis for mapping")

# Load the large model with vector embeddings based on scientific text
nlp = spacy.load("en_core_sci_lg")

# Adding the UMLS entity linker
nlp.add_pipe("scispacy_linker", config={"resolve_abbreviations": True, "linker_name": "umls"})

print("Model loaded successfully!")

# Initialize a list to hold extracted entities
entities = []

# Process each row in the diagnosis dataset
for index, row in df_transformed.iterrows():
    subject_id = row["subject_id"]  # Include subject ID
    hadm_id = row["hadm_id"]  # Hospital admission ID
    diagnosis_text = row["discharge_diag"]  # Discharge diagnosis

    # Apply NLP model to the full diagnosis text
    doc = nlp(diagnosis_text)

    # Extract UMLS concepts for the entire diagnosis
    for ent in doc.ents:
        for umls_ent in ent._.kb_ents:
            linker = nlp.get_pipe("scispacy_linker")
            concept = linker.kb.cui_to_entity[umls_ent[0]]
            
            score = umls_ent[1]  # Confidence score
            
            # Append extracted information
            entities.append({
                "Subject_ID": subject_id,  # Maintain Subject ID
                "hadm_id": hadm_id,  # Maintain the ID from the CSV
                "Diagnosis": diagnosis_text,  # Keep full diagnosis text
                "UMLS_CUI": concept.concept_id,
                "UMLS_Name": concept.canonical_name,
                "Score": score  # Keep the matching score
            })

# Convert extracted entities into a DataFrame
entities_df = pd.DataFrame(entities)
print("Entities extracted successfully!")

#entities_df.head(10)

# Rename "cui" to "UMLS_CUI" for consistency in umls-icd10 CSV file to match
code_df.rename(columns={"cui": "UMLS_CUI"}, inplace=True) 


# Merge with UMLS-ICD10 mapping
merged_df = entities_df.merge(code_df, on="UMLS_CUI", how="left")

# Select the row with the highest Score for each Diagnosis
best_matches_df = merged_df.loc[
    merged_df.groupby(["Subject_ID", "hadm_id", "Diagnosis"])["Score"].idxmax()
]

best_matches_df.head(10)

print(f"Total rows processed: {len(best_matches_df)}")

#save to csv
best_matches_df.to_csv("structured_diagnosis_mappings")
print("saved_successfully")

UMLS-ICD10 File loaded successfully!
loaded diagnosis for mapping
Model loaded successfully!
Entities extracted successfully!
Total rows processed: 1016026
saved_successfully


In [24]:
best_matches_df.head(10)

,Subject_ID,hadm_id,Diagnosis,UMLS_CUI,UMLS_Name,Score,code,str,tty,sab
0,10000032,22595853,Ascites from Portal HTN,C0003962,Ascites,0.988981,R18,Ascites,AB,ICD10CM
17,10000032,22841357,"HCV cirrhosis, HIV, hyponatremia, COPD",C0020625,Hyponatremia,0.989849,E87.1,Sodium [Na] deficiency,ET,ICD10CM
5,10000032,22841357,diuretic refractory ascites,C0003962,Ascites,0.988981,R18,Ascites,AB,ICD10CM
46,10000032,25742920,Decompensated HCV cirrhosis,C0205434,Decompensated,0.972250,5650002,Decompensated,PT,SNOMEDCT_US
53,10000032,25742920,Hyponatremia,C0020625,Hyponatremia,0.989849,E87.1,Sodium [Na] deficiency,ET,ICD10CM
30,10000032,29079034,Acute Kidney Injury,C0022660,"Kidney Failure, Acute",0.975594,N17.9,"Acute kidney failure, unspecified",PT,ICD10CM
45,10000032,29079034,COPD,C0024117,Chronic Obstructive Airway Disease,0.919293,J44.9,"Chronic obstructive pulmonary disease, unspeci...",PT,ICD10CM
40,10000032,29079034,Cirrhosis,C0023890,Liver Cirrhosis,0.976766,K74.60,Cirrhosis (of liver) NOS,ET,ICD10CM
35,10000032,29079034,HIV,C0019682,HIV,0.988085,19030005,Human immunodeficiency virus,PT,SNOMEDCT_US
26,10000032,29079034,Hyperkalemia,C0020461,Hyperkalemia,0.960652,E87.5,Hyperkalemia,PT,ICD10CM


In [19]:
##PART 4 GET COUNTS AFTER TRANSFORMATION
print(len(df_transformed))

print( df_transformed['subject_id'].nunique() ) 

print( df_transformed['hadm_id'].nunique() ) 

print( df_transformed['discharge_diag'].nunique() ) 

1079091
145351
321419
405159


In [21]:
print(len(entities_df))

print( entities_df['Subject_ID'].nunique() ) 

print( entities_df['hadm_id'].nunique() ) 

print( entities_df['Diagnosis'].nunique() ) 

5503294
142601
313343
372340


In [23]:
##PART 4 GET COUNTS AFTER TRANSFORMATION
print(len(best_matches_df))

print( best_matches_df['Subject_ID'].nunique() ) 

print( best_matches_df['hadm_id'].nunique() ) 

print( best_matches_df['Diagnosis'].nunique() ) 



1016026
142601
313343
372340
